# Pandas and SQL
Structured Query Language, or **SQL**, is the language of databases. Though "SQL" may seem new to you, you've actually used it in your GIS work, specifically in selecting records by attribute values. SQL, however, is capable of much more than just selecting records; SQL is used to create, read, relate (i.e. join), update, delete records stored in database tables. 

In this exercise we use Pandas to mimic many SQL functions, showing that we don't need to learn daunting database management systems like Oracle, Microsoft SQL Server, PostgreSQL, or even MS Access, to run some of the everyday database functions you might want to run. Instead, we can import data as a dataframe and use some Pandas functions to get what you need. 

Here we explore some useful data query and transformation techniques are applied to marine species observations data extracted from OBIS/SeaMap. These data are stored in the data folder as `Seamap400.csv`. 

→ *You may want to examine this file in Excel to familiarize yourself with the data first.*

In [1]:
#Import the modules
import pandas as pd
import numpy as np #Well need NumPy for some of it's data types...

## Loading the csv data into a Pandas dataframe
We've loaded CSV data into a Pandas dataframe in previous exercises. The code block below demonstrates how the `read_csv` function easily does this. 

In [2]:
#Import the data into a dataframe called "dfSeamap"
dfSeamap = pd.read_csv('Seamap400.csv')
#Show the first 2 records in the result
dfSeamap.head(2)

,ID,datasetid,rowid,last_mod,latitude,longitude,sp_code,sp_tsn,sp_count,obs_date,scientific,sp_common,sp_class,provider
0,1,400,1114,6/15/2007 10:06:59,37.27,-75.20,BALA,180522,2,3/13/2007 14:11:00,Balaenopteridae,rorquals,Mammalia,University of North Carolina at Wilmington
1,2,400,1006,6/15/2007 10:06:59,36.95,-75.87,Bph,180527,2,2/28/2007 10:52:00,Balaenoptera physalus,Fin Whale,Mammalia,University of North Carolina at Wilmington


The result *looks* good, and indeed we can do much with this dataframe. BUT, there are some "gotchas" that we can fix:
* First, Pandas assigns it's own index, when we might want to keep the ID field as the index values.
* The `dataset` and `rowid` fields, are actually nominal values, not actual numbers, and we should import these as strings, not numbers (in case they have a leading zero, for example).
* The `obs_date` field is imported as a string, but we can convert this as a date object, which will allow for selection via time slices.


So... to fix this, we can specify a few extra parameters when importing the data. Below is the `read_csv()` function applies as above, but with three additional parameters:
* The `index_col` specifies which column will be our dataFrame's **index**. 
* `parse_dates` indicates that the `last_mod` and `obs_date` columns are a **date fields**, not strings, which enables us to to time-based queries. 
* And finally, the `dtype` functions allows us to override the defaut data type assignments. Here, we pass a dictionary consisting of column names (keys) and data types (values) to indicate which columns:data type pairs we want to specifiy. 

In [3]:
#Import the Seamap400.csv file to a DataFrame
dfSeamap = pd.read_csv('Seamap400.csv', 
                       index_col = 'ID',
                       parse_dates=['last_mod','obs_date'],
                       dtype={'datasetid':'str',
                              'rowid':'str'
                             }
                      )
dfSeamap.head(2)

,datasetid,rowid,last_mod,latitude,longitude,sp_code,sp_tsn,sp_count,obs_date,scientific,sp_common,sp_class,provider
ID,,,,,,,,,,,,,
1,400,1114,2007-06-15 10:06:59,37.27,-75.20,BALA,180522,2,2007-03-13 14:11:00,Balaenopteridae,rorquals,Mammalia,University of North Carolina at Wilmington
2,400,1006,2007-06-15 10:06:59,36.95,-75.87,Bph,180527,2,2007-02-28 10:52:00,Balaenoptera physalus,Fin Whale,Mammalia,University of North Carolina at Wilmington


Other than the ID now being used as the index the result looks the same, but if you inspect the data types, you'll see we got what we wanted.

In [4]:
#Check the data types of the import
dfSeamap.dtypes

datasetid             object
rowid                 object
last_mod      datetime64[ns]
latitude             float64
longitude            float64
sp_code               object
sp_tsn                 int64
sp_count               int64
obs_date      datetime64[ns]
scientific            object
sp_common             object
sp_class              object
provider              object
dtype: object

## Exploring our dataFrame
Now, let's examine how many unique species there are. This is analogous to the `SELECT DISTINCT` SQL operation.

In [5]:
dfSeamap.sp_common.unique()

array(['rorquals', 'Fin Whale', 'Loggerhead', 'cartilaginous fishes',
       'Basking Shark', 'Common Dolphin', 'dolphins',
       'Northern Right Whale', 'Short-finned Pilot Whale',
       'Ocean Sunfish', 'Atlantic Ridley', 'Atlantic Manta',
       'Humpback Whale', 'cow-nose rays', 'Bottlenose Dolphin',
       'Sea Turtles'], dtype=object)

## Selecting <u>columns</u> of data from a dataFrame
Selecting columns is analogous to the SQL `SELECT ... FROM` clause, but the syntax in Pandas is a bit different.

#### ♦ Selecting data from <u>a single column</u> in the dataframe
Selecting a single column is easy as specifying the column name, either between brackets with the field name in quotes, or using the dot notation. (*The former is preferred, in case your field name is the the same as another property or method of a dataFrame.*)

When only one column is selected, the object returned is actually not a dataFrame, but a pandas **series** object, which is quite similar to a one-dimensional NumPy array. (However, in Pandas, series are limited to one dimension...)

In [17]:
#Select just the sp_common field into a new view
commonNames = dfSeamap['sp_common']
type(commonNames)

pandas.core.series.Series

In [7]:
#Using "dot notation" (<dataframe>.<field name>) also works
commonNames = dfSeamap.sp_common
type(commonNames)

pandas.core.series.Series

In [8]:
#As the object returned is a series, we can get records from its index
commonNames[:8] #Returns the first 8 records

ID
1     rorquals
2    Fin Whale
3    Fin Whale
4    Fin Whale
5    Fin Whale
6    Fin Whale
7    Fin Whale
8    Fin Whale
Name: sp_common, dtype: object

In [ ]:
#What code would you use to get the last 8 records? (Replace the ???)
commonNames[???]

#### ♦ Selecting data  from <u>multiple</u> columns
Selecting multiple columns can be done by including a list of the column names. *Note the double set of brackets: the outer set is the syntax of the dataFrame object, and the inner set denotes the list of field names we want.*

Here, the object returned resembles a dataFrame, not a series (since Pandas' series can only be one-dimension). However, in actuality, this is simply a **view** of the original dataFrame, not a new object - much like query views in SQL...

In [9]:
#Select the 'sp_common' and 'sp_class' columns into a new view
dfSelect = dfSeamap[['sp_common','sp_class']]
dfSelect.head()

,sp_common,sp_class
ID,,
1,rorquals,Mammalia
2,Fin Whale,Mammalia
3,Fin Whale,Mammalia
4,Fin Whale,Mammalia
5,Fin Whale,Mammalia


In [10]:
##What code would create a table of just the species common name and the latitude and longitude columns?
dfSelect2 = dfSeamap[[???]]
dfSelect2.head()

,sp_common,latitude,longitude
ID,,,
1,rorquals,37.27,-75.20
2,Fin Whale,36.95,-75.87
3,Fin Whale,36.96,-75.43
4,Fin Whale,36.95,-75.49
5,Fin Whale,36.70,-75.55


## Selecting <u>rows</u> in a dataFrame
Selecting rows, analogous to the `select...where` statement in SQL, can be done by creating **boolean masks** of the criteria you want and then applying those masks. This can be done explicitily as two steps, or more commonly in a single, compount step. 

Before making our selection, let's examine how we can easily extract a list of valid options using the `unique` property.

In [11]:
#List the unique values in the 'sp_class' field.
dfSeamap['sp_class'].unique()

array(['Mammalia', 'Reptilia', 'Chondrichthyes', 'Actinopterygii'],
      dtype=object)

In [12]:
#Display just the *number* of unique latitude values
dfSeamap['latitude'].nunique()

267

#### Selecting rows in two steps:

In [13]:
#Step 1: Create the boolean mask
theMask = dfSeamap['sp_class'] == 'Mammalia'
theMask[2150:2156] #Show a selection of records, to display values

ID
2151     True
2152     True
2153     True
2154    False
2155    False
2156    False
Name: sp_class, dtype: bool

In [14]:
#Step 2: Applying the mask to return only 'true' records
dfMammals = dfSeamap[theMask]
dfMammals.sample(4) #Show a random sample from the product

,datasetid,rowid,last_mod,latitude,longitude,sp_code,sp_tsn,sp_count,obs_date,scientific,sp_common,sp_class,provider
ID,,,,,,,,,,,,,
2059,400,1972,2007-06-15 10:06:59,35.27,-75.35,Ttr,180426,5,2007-04-24 11:28:00,Tursiops truncatus,Bottlenose Dolphin,Mammalia,University of North Carolina at Wilmington
1836,400,1443,2007-06-15 10:06:59,35.53,-75.17,Ttr,180426,1,2007-03-22 14:13:00,Tursiops truncatus,Bottlenose Dolphin,Mammalia,University of North Carolina at Wilmington
2013,400,1890,2007-06-15 10:06:59,35.20,-75.54,Ttr,180426,1,2007-04-23 09:32:00,Tursiops truncatus,Bottlenose Dolphin,Mammalia,University of North Carolina at Wilmington
2,400,1006,2007-06-15 10:06:59,36.95,-75.87,Bph,180527,2,2007-02-28 10:52:00,Balaenoptera physalus,Fin Whale,Mammalia,University of North Carolina at Wilmington


In [15]:
#Proof that we only got mammals in the result
dfMammals['sp_class'].unique()

array(['Mammalia'], dtype=object)

#### Selecting rows in one step:

In [18]:
#Get all the mammals and report all unique scientific names
dfMammals = dfSeamap[dfSeamap['sp_class'] == 'Mammalia']
dfMammals.shape #Reveal the size of the result

(929, 13)

In [19]:
dfMammals['scientific'].unique()

array(['Balaenopteridae', 'Balaenoptera physalus', 'Delphinus delphis',
       'Delphinidae', 'Eubalaena glacialis', 'Globicephala macrorhynchus',
       'Megaptera novaeangliae', 'Tursiops truncatus'], dtype=object)

#### Selecting rows using `query`:
We can also execute familiar **query strings** to extract records. *Personally, I don't like this method as the query strings tend to be finicky and can be tough to use with quotes..*

In [20]:
dfMammals = dfSeamap.query("sp_class == 'Mammalia'")
dfMammals.shape

(929, 13)

#### Complex row selections
We can combine masks using logical operators (`&` = "and", `|` = "or").

In [21]:
#Select mammal records in the 2nd half of 2006
mammalMask = dfSeamap['sp_class'] == 'Mammalia'

#Create date objects for the start and end dates
startDate = np.datetime64('2006-07-01')
endDate = np.datetime64('2007-01-01')

#Create the date masks
startMask = (dfSeamap['obs_date'] >= startDate) 
endMask = (dfSeamap['obs_date'] < endDate)

#Apply the masks, using the bitwise '&' to return rows where all masks are true
dfSelect2 = dfSeamap[mammalMask & startMask & endMask]
dfSelect2['scientific'].unique()

array(['Delphinus delphis', 'Delphinidae', 'Eubalaena glacialis',
       'Tursiops truncatus'], dtype=object)

#### Return rows matching a substring
The `.str` function on a column allows us to use some string operations  on the values in that field.  Here we use the string `startswith` function to return all rows where the row's value starts with 'Delphin'. See https://pandas.pydata.org/pandas-docs/stable/text.html for other string operations.

In [ ]:
#Select rows where the scientific name starts with "Delphin" 
dolphinMask = dfSeamap['scientific'].str.startswith('Delphin')
dfDolphins = dfSeamap[dolphinMask]

#Use the nunique function to just return the number 
#  of unique scientific names
dfDolphins['scientific'].nunique()

#### More complex queries with `apply` and  `lambda`
For maximum flexibility, we can actually write our own functions to be applied to each value in a column (or multiple columns). This is done using the `apply` function to a dataFrame and then specifying the subcode we want to use with Python's `lambda` statement. (This seems fairly complex at first, but it actually somewhat straightforward -- and can be very useful...)

In [ ]:
#Create a mask by searching each row for the string "Whale" and apply the mask
# to list the scientific names of these records
whaleMask = dfSeamap['sp_common'].apply(lambda x: 'Whale' in x)
dfWhale = dfSeamap[whaleMask]
dfWhale['scientific'].unique()

## Grouping/Aggregating data
Pandas can aggregate data on values like SQL as well. We do this with the `groupby` statement

In [ ]:
#Count the observations by common name
grpSpCommon = dfSeamap.groupby('sp_common')
grpSpCommon['sp_common'].count()

We can also show *all* the summary stats with the dataFrame's `describe` function.

In [ ]:
grpSpCommon.describe()

## Reshaping tables
In addition to grouping data, we can "pivot" the data, summarizing by one field and aggregating values across other fields. Let's look at an example where we display 

In [ ]:
#Extract just the dolphin records
#dfDolphin = dfSeamap.query("sp_common == 'dolphins'").copy(deep=True)
dfDolphin = dfSeamap.copy(deep=True)

In [ ]:
#Add a "day" column (https://pandas.pydata.org/pandas-docs/stable/api.html#datetimelike-properties)
dfDolphin['day'] = dfDolphin['obs_date'].dt.weekofyear
dfDolphin.head()

In [ ]:
#Pivot to a table listing row=species, col = day of year, values = latitude
dfLat = dfDolphin.pivot_table(index='sp_common',columns='day',values='latitude')
dfLat.head()

## Joining tables
Also like SQL, pandas can join tables. Below we'll create two tables from our aggregated data: one will list the minimum of the latitude and longitude columns, and the second will list the maximum values. Then we'll join these two tables and compute the geographic extent of each species observations. 

In [ ]:
#Create dataFrames of the minimum and then maximum of the lat and lng fields
minCoords = grpSpCommon['latitude','longitude'].min()
maxCoord = grpSpCommon['latitude','longitude'].max()

In [ ]:
# Have a look at what is returned (for the min table)
minCoords.head()

In [ ]:
#Use the Pandas 'merge' command to join the two tables
sppExtent = pd.merge(left=minCoords,     #Specifies the left table
                     right=maxCoord,     #Specifies the right table
                     how = 'inner',      #Specifies the type of join
                     left_index=True,    #Use the index of the left table as the join item
                     right_index=True)   #Use the index of the right table as the join item
#Have a look
sppExtent.head()

In [ ]:
#Rename the columns to the values in the list provided
sppExtent.columns = ['minX','minY','maxX','maxY']
sppExtent.head()

In [ ]:
#Compute two new columns as the difference between max and min
sppExtent['XRange'] = sppExtent.maxX - sppExtent.minX
sppExtent['YRange'] = sppExtent.maxY - sppExtent.minY
sppExtent.head()